# Microblaze RPC Bindings

In addition to the `%%microblaze` magic there is also a set of Python bindings for the core API. These bindings are designed to provide an easy to use way of making use of the APIs with the full functionality of Python available. As the API is as close to identical as possible to the C API, any performance critical code can be moved to C with minimal effort.

The two classes are `MbioBinding` which provides just the core `mbio.h` functions and `IopSwitchBinding` which provides the additional functionality from the `iop_switch.h` functions for use with the PMOD and Arduino IOPs.

The constructors for both take the IOP to run on and optionally an array of modules that can provide additional functionality. For this example we are going to concentrate on the `IopSwitchBinding` as we are using the Base overlay PMOD and Arduino IOPs.

First step is to load the Base Overlay and create an instance of `IopSwitchBinding` on PMODA

In [8]:
from pynq.overlays.base import BaseOverlay
from ipython_microblaze import IopRPC

base = BaseOverlay('base.bit')
iop = IopRPC(base.PMODA)

We can now inspect the `iop` variable to find all of the functions that have been wrapped.

In [9]:
dir(iop)

['G1',
 'G2',
 'G3',
 'G4',
 'GPIO_IN',
 'GPIO_OUT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 '_rpc_stream',
 'analog_num_devices',
 'analog_open',
 'analog_open_iop_grove',
 'analog_open_iop_pin',
 'analog_range',
 'analog_read',
 'analog_read_raw',
 'analog_vref',
 'delay',
 'delayMicroseconds',
 'gpio_num_devices',
 'gpio_open_all',
 'gpio_open_iop_grove',
 'gpio_open_iop_pin',
 'gpio_open_pin',
 'gpio_open_range',
 'gpio_read',
 'gpio_set_direction',
 'gpio_write',
 'i2c_num_devices',
 'i2c_open',
 'i2c_open_iop_grove',
 'i2c_open_iop_pins',
 'i2c_read',
 'i2c_write',
 'mp_spi_open',
 '

You can see that all of the functions from the other notebooks are here along with the enum constants. We can now use these functions to turn on an LED attached to G2

In [10]:
led = iop.gpio_open_iop_grove(iop.G2, 0)
iop.gpio_set_direction(led, iop.GPIO_OUT)
iop.gpio_write(led, 1)

To make the API slightly easier to use in python, each `typedef` in the original source is wrapped as a custom python class. Any methods with that type name as prefix are then added as member functions. Using this API we can write a flashing script in a more python-like way

In [4]:
import time
for _ in range(10):
    led.write(0)
    time.sleep(0.2)
    led.write(1)
    time.sleep(0.2)

## Pointer arguments

A more complex API is that of the I2C controller which has to send and receive variable strings of bytes. In Python these are `bytes` objects for const strings or `bytearrays` for non-const strings. Other array types of size 1-byte may also work but haven't been thoroughly tested yet. Copying the example from a previous notebook, let's read from an I2C attached ADC. The original C code looked like:

```C
#define ADDRESS 0x50

i2c bus = i2c_connect_grove(G4);
unsigned char data[2];
// Setup the device
data[0] = 0x02;
data[1] = 0x20;
i2c_write(bus, ADDRESS, data, 2);
// Read the value
data[0] = 0x00;
i2c_write(bus, ADDRESS, data, 1);
i2c_read(bus, ADDRESS, data, 2);
```

In [5]:
ADDRESS = 0x50

bus = iop.i2c_open_iop_grove(iop.G4)
data = bytearray(b'\x02\x20')
bus.write(ADDRESS, data, 2)
data[0] = 0x00
bus.write(ADDRESS, data, 1)
bus.read(ADDRESS, data, 2)

data

bytearray(b'\x00P')

More complex peripherals are exposed in C as modules. Modules can be added into the set of bindings by passing the names as an additional array argument. For this example we are going to use the Grove LED bar library as it comes with the ipython_microblaze package. Before changing the bindings we need to reset the microblaze.

In [6]:
iop.reset()
iop = IopRPC(base.PMODA, ['ledbar'])

If we now look at the members of the `iop` variable again we will see that the `ledbar_*` functions now appear.

In [7]:
dir(iop)

['G1',
 'G2',
 'G3',
 'G4',
 'GPIO_IN',
 'GPIO_OUT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 'analog_num_devices',
 'analog_open',
 'analog_open_iop_grove',
 'analog_open_iop_pin',
 'analog_range',
 'analog_read',
 'analog_read_raw',
 'analog_vref',
 'delay',
 'delayMicroseconds',
 'gpio_num_devices',
 'gpio_open_all',
 'gpio_open_iop_grove',
 'gpio_open_iop_pin',
 'gpio_open_pin',
 'gpio_open_range',
 'gpio_read',
 'gpio_set_direction',
 'gpio_write',
 'i2c_num_devices',
 'i2c_open',
 'i2c_open_iop_grove',
 'i2c_open_iop_pins',
 'i2c_read',
 'i2c_write',
 'ledbar_open',
 'ledbar_open_grov

We can now use this API to make an increasing pattern appear on the bar. Same as for the `mb_*` typedefs, the `ledbar_` functions all appear as members of an `ledbar` class.

In [8]:
bar = iop.ledbar_open_grove(iop.G1)
for i in range(11):
    bar.set_level(i)
    time.sleep(0.5)

## Custom RPC Functions

The bindings are generated dynamically from the C sources and it possible to use other files as the input. The `MicroblazeRPC` class exposes this functionality for general sources. The current restrictions on functions used are:

 * No structs or unions in the interface
 * Only const and non-const char* pointer arguments
 * No pointer return types
 * Typedefs for classes must resolve to basic types
 * No printing to stout
 
Subject to these restrictions, any code can be run in this environment for experimentation and debugging.

The `MicroblazeRPC` class allows for the construction of RPC interfaces based on any C code that meets these restrictions. Note that only functions with a separate declaration will be pulled into the interface.

In [9]:
from ipython_microblaze import MicroblazeRPC

program_text = """
int add(int a, int b);
void memcpy(const char* source, char* dest, int length);

void memcpy(const char* source, char* dest, int length){
    while (length-- > 0) {
        *dest++ = *source++;
    }
}
void int_helper(int a, int b, int* c) {
    *c = a + b;
}

int add(int a, int b) {
    int result;
    int_helper(a, b, &result);
    return result;
}
"""

custom = MicroblazeRPC(base.PMODB, program_text)

dir(custom)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_constants',
 '_build_functions',
 '_mb',
 '_populate_typedefs',
 'add',
 'memcpy',
 'reset',
 'visitor']

As can be seen only the `add` and `memcpy` functions have been added to rpc layer. We can now call them as per any of the other examples.

In [10]:
print(custom.add(2,4))
source = bytes(range(16))
dest = bytearray(16)

custom.memcpy(source, dest, 16)
print(dest)

6
bytearray(b'\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\x0c\r\x0e\x0f')


To have custom classes created, create a typedef of the class name and prefix all of your methods with that typedef. If the first parameter to the function is of that type then it is used as `self` when called, otherwise the method with be static.

In [11]:
class_text = """
typedef int my_type;

my_type my_type_construct(int val);
int my_type_add(my_type this, int other);

my_type my_type_construct(int val) { return val; }
int my_type_add(my_type this, int other) { return this + other; }
"""

type_rpc = MicroblazeRPC(base.ARDUINO, class_text)

In [12]:
mine = type_rpc.my_type_construct(4)
print(mine.add(7))

11


Note that if you `#include` a file then all functions inside it which meet these guidelines will also be added to the resulting RPC instance. This makes it a convenient way of assembling a set of modules with a small amount of custom code. This approach can also add new methods to the dynamic classes.

In [13]:
combined_text = """
#include <ledbar.h>
#include <iop.h>

void ledbar_clear(ledbar bar);
void ledbar_clear(ledbar bar) {
    ledbar_set_level(bar, 0);
}
"""

iop.reset()
iop = MicroblazeRPC(base.PMODA, combined_text)

In [14]:
bar = iop.ledbar_open_grove(iop.G1)
bar.clear()

It's advised to keep any module APIs following the guidelines for use with RPC so that other uses can use your module from both Python and C. Over time these restrictions will become less onerous (hopefully)

In [5]:
led.val

1

In [7]:
iop._rpc_stream.read_channel.control_array

array([1016,    2], dtype=uint32)